In [3]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6


Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 4.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=a158df20891fcb4e4fcc8c1ccb3495a81857851d016020e8b18deab0bf55e5be
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [6]:
!kaggle -v


Kaggle API 1.5.6


In [7]:
!kaggle competitions download -c regression-cabbage-price


  0% 0.00/29.7k [00:00<?, ?B/s]
100% 29.7k/29.7k [00:00<00:00, 24.7MB/s]


In [8]:
!unzip regression-cabbage-price.zip


Archive:  regression-cabbage-price.zip
  inflating: submit_sample.csv       
  inflating: test_cabbage_price.csv  
  inflating: train_cabbage_price.csv  


In [241]:
import torch
import torch.optim as optim 
import numpy as np 
import math
import torch.nn.functional as F


torch.manual_seed(1)

data = np.loadtxt('train_cabbage_price.csv',delimiter = ',',dtype=np.float32,skiprows=1,usecols=range(1,6))
x_data = torch.from_numpy(data[:,1:-2]) 
y_data = torch.from_numpy(data[:,[-1]])

data_test = np.loadtxt('test_cabbage_price.csv',delimiter = ',',dtype=np.float32,skiprows=1,usecols=range(1,5))

test_x_data = torch.from_numpy(data_test[:,1:-1])



print(x_data)
print(test_x_data)


tensor([[-11.0000,   0.9000],
        [ -5.5000,   5.5000],
        [ -6.9000,   1.4000],
        ...,
        [ -2.1000,   8.0000],
        [ -1.6000,   7.1000],
        [ -2.0000,   5.8000]])
tensor([[ 8.0000, 19.2000],
        [-3.0000,  9.5000],
        [ 5.1000, 19.6000],
        [11.9000, 22.5000],
        [17.8000, 27.4000],
        [13.9000, 24.4000]])


In [265]:
w = torch.zeros((2,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([w,b],lr=0.000005,momentum=0.9) 

nb_epochs = 250000
#minibatch_size = 20

for epoch in range(nb_epochs + 1):
    hypothesis = x_data.matmul(w)+b
    cost = torch.mean((hypothesis-y_data) **2)
    
    #옵티마이저 초기화
    optimizer.zero_grad()
    # cost 계산 
    cost.backward()
    # 옵티마이저 갱신 
    optimizer.step()

    if epoch % 10000 == 0:
      print('epoch {:4d}/{}  cost:{:.6f} rmse = {:.6f}'.format(
          epoch,nb_epochs,cost.item(),math.sqrt(cost.item())
      ))

epoch    0/250000  cost:12610017.000000 rmse = 3551.058575
epoch 10000/250000  cost:2322144.500000 rmse = 1523.858425
epoch 20000/250000  cost:2258450.500000 rmse = 1502.814193
epoch 30000/250000  cost:2202940.750000 rmse = 1484.230693
epoch 40000/250000  cost:2154564.250000 rmse = 1467.843401
epoch 50000/250000  cost:2112402.750000 rmse = 1453.410730
epoch 60000/250000  cost:2075661.375000 rmse = 1440.715577
epoch 70000/250000  cost:2043637.500000 rmse = 1429.558498
epoch 80000/250000  cost:2015736.000000 rmse = 1419.766178
epoch 90000/250000  cost:1991414.375000 rmse = 1411.174821
epoch 100000/250000  cost:1970220.000000 rmse = 1403.645254
epoch 110000/250000  cost:1951745.000000 rmse = 1397.048675
epoch 120000/250000  cost:1935646.500000 rmse = 1391.275135
epoch 130000/250000  cost:1921621.000000 rmse = 1386.225451
epoch 140000/250000  cost:1909389.000000 rmse = 1381.806426
epoch 150000/250000  cost:1898735.000000 rmse = 1377.945935
epoch 160000/250000  cost:1889448.875000 rmse = 13

In [266]:
x_test = torch.tensor([-6.6,2.])
value = x_test.matmul(w)+b
print(value)
y_val = x_data.matmul(w)+b
print(y_val)
correct_pred = y_val.float()-y_data
print(correct_pred)


tensor([2719.7061], grad_fn=<AddBackward0>)
tensor([[2795.6812],
        [2899.1130],
        [2692.5884],
        ...,
        [2941.9124],
        [2870.2212],
        [2803.3047]], grad_fn=<AddBackward0>)
tensor([[672.6812],
        [776.1130],
        [569.5884],
        ...,
        [ 40.9124],
        [-30.7788],
        [-97.6953]], grad_fn=<SubBackward0>)


In [267]:
prediction = test_x_data.matmul(w)+b
 # test_x_data.matmul(w)+b
print(prediction)


tensor([[3301.1978],
        [3063.5742],
        [3420.4817],
        [3376.8892],
        [3485.7180],
        [3428.4824]], grad_fn=<AddBackward0>)


In [268]:
import pandas as pd

submit = pd.read_csv('submit_sample.csv')
submit

,Id,Expected
0,0,1000
1,1,1000
2,2,1000
3,3,1000
4,4,1000
5,5,1000


In [269]:
submit['Expected'][0]

1000

In [270]:
len(prediction)

6

In [0]:
for i in range(len(prediction)):
    submit['Expected'][i] = prediction[i].item()

In [274]:
submit

,Id,Expected
0,0,3301
1,1,3063
2,2,3420
3,3,3376
4,4,3485
5,5,3428


In [0]:
submit.to_csv('submit.csv',mode='w', index=False)

In [276]:
!kaggle competitions submit -c regression-cabbage-price -f submit.csv -m "14010974_이기택"

100% 54.0/54.0 [00:02<00:00, 25.0B/s]
403 - Your team has used its submission allowance (20 of 20). This resets at midnight UTC (17 hours from now).
